In [1]:
import syft as sy
from syft.store.blob_storage import BlobStorageConfig, BlobStorageClientConfig
from syft.store.blob_storage.seaweedfs import SeaweedFSClient, SeaweedFSClientConfig
from syft import ActionObject
from syft.service.action.action_data_empty import ActionFileData
from syft.service.queue.zmq_queue import ZMQQueueConfig, ZMQClientConfig
from collections import defaultdict

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft/notebooks


In [2]:
node = sy.orchestra.launch(name="test-domain-helm2", dev_mode=True,
                           reset=True,
                           n_consumers=0,
                           create_producer=True)
client = node.login(email="info@openmined.org", password="changethis")

Logged into <test-domain-helm2: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [3]:
client.worker.start_workers(n=3)

> /Users/koen/workspace/PySyft/packages/syft/src/syft/service/worker/worker_service.py(52)get_default_env_vars()
     50     import pdb
     51     pdb.set_trace()
---> 52     if context.node.runs_in_docker:
     53         # get env vars from current environment
     54         return dict(os.environ)

ipdb> c
> /Users/koen/workspace/PySyft/packages/syft/src/syft/service/worker/worker_service.py(52)get_default_env_vars()
     50     import pdb
     51     pdb.set_trace()
---> 52     if context.node.runs_in_docker:
     53         # get env vars from current environment
     54         return dict(os.environ)

ipdb> q


SyftError: Exception calling worker.start_workers. Traceback (most recent call last):
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/node/node.py", line 929, in handle_api_call_with_unsigned_result
    result = method(context, *api_call.args, **api_call.kwargs)
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/service/service.py", line 339, in _decorator
    result = func(self, *args, **kwargs)
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/service/worker/worker_service.py", line 152, in start_workers
    res = start_worker_container(context)
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/service/worker/worker_service.py", line 125, in start_worker_container
    env_vars = get_env_vars(context)
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/service/worker/worker_service.py", line 84, in get_env_vars
    default_env_vars = get_default_env_vars(context)
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/service/worker/worker_service.py", line 52, in get_default_env_vars
    if context.node.runs_in_docker:
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/service/worker/worker_service.py", line 52, in get_default_env_vars
    if context.node.runs_in_docker:
  File "/opt/anaconda3/envs/syft/lib/python3.9/bdb.py", line 88, in trace_dispatch
    return self.dispatch_line(frame)
  File "/opt/anaconda3/envs/syft/lib/python3.9/bdb.py", line 113, in dispatch_line
    if self.quitting: raise BdbQuit
bdb.BdbQuit

In [7]:
workers = client.worker.list()

In [8]:
workers

[syft.service.worker.worker_service.DockerWorker,
 syft.service.worker.worker_service.DockerWorker,
 syft.service.worker.worker_service.DockerWorker]

In [6]:
client.worker.stop(workers)

SyftSuccess: 2 workers stopped

## DEV

In [8]:
import docker

In [12]:
image = "openmined/grid-backend"
client = docker.from_env()

In [13]:
PYSYFT_DIR = f"/Users/koen/workspace/pysyft"

In [14]:
volumes = {
    f'{PYSYFT_DIR}/packages/syft': {'bind': '/app/syft', 'mode': 'rw'},
    f'{PYSYFT_DIR}/packages/grid/backend/grid': {'bind': '/app/grid', 'mode': 'rw'},
    "/var/run/docker.sock": {"bind": "/var/run/docker.sock"}

}

In [15]:
def get_default_env_vars():
    env_path = f"{PYSYFT_DIR}/packages/grid/.env"

    with open(env_path, "r") as f:
        lines = f.read().splitlines()

    default_env_vars = dict()
    for l in lines:
        if "=" in l:
            try:
                var_name, value = l.split("=", 1)
                
                def remove_redundant_quotes(value):
                    for s in ['"', "'"]:
                        if len(value) !=0:
                            if value[0] == s:
                                value = value[1:]
                            if value[-1] == s:
                                value = value[:-1]
                                
                value = remove_redundant_quotes(value)
                default_env_vars[var_name] = value
            except Exception as e:
                print("error parsing env file", e)
    return default_env_vars

In [16]:
def get_env_vars():
    default_env_vars = get_default_env_vars()
    import secrets
    worker_tag = "".join([str(secrets.choice(list(range(10)))) for i in range(10)])

    extra_env_vars = {
        "SERVICE_NAME": "backend",
        "CREATE_PRODUCER": "false",
        "N_CONSUMERS": "1",
        "DEV_MODE": "True",
        "DEFAULT_ROOT_USERNAME": f"worker-{worker_tag}",
        "PORT": "8003",
        "HTTP_PORT": "88",
        "HTTPS_PORT": "446",
        "DEFAULT_ROOT_EMAIL": f"{worker_tag}@openmined.org"
        
    }

    return {**default_env_vars, **extra_env_vars}
    

In [17]:
import random

In [18]:
random.randint(0, 1000)

220

In [19]:
env_vars = get_env_vars()

In [20]:
from pprint import pprint

In [21]:
from grid.backend.core.config import settings

ModuleNotFoundError: No module named 'grid'

In [22]:
from pathlib import Path
str(Path("/Users/koen/workspace/pysyft/packages/grid").parent.parent)

'/Users/koen/workspace/pysyft'

In [23]:
# pprint(env_vars)

In [24]:
# env_vars

In [25]:
# client.containers

In [26]:
# client.containers.run('alpine', 'echo hello world', )

In [31]:
res = client.containers.run(image=image, volumes=volumes, environment=env_vars,
                            detach=True, network_mode="container:grid-proxy-1")

In [32]:
res.id

'b7a8e3c2a0b8295d22bebdcd004d7a2f14b05c187c4b3cc20a37e914938aee63'

In [118]:
for x in client.containers.list(filters={"id": res.id}):
    x.stop()

In [123]:
x = res.stop()

In [121]:
x

In [50]:
# print(res.logs().decode())

In [62]:
# env_vars["BACKEND_CORS_ORIGINS"].replace("\'", "'")

In [56]:
["http://localhost","http://localhost:4200","http://localhost:3000","http://localhost:8080","https://localhost","https://localhost:4200","https://localhost:3000","https://localhost:8080","http://dev.grid.openmined.org","https://stag.grid.openmined.org","https://grid.openmined.org"]

['http://localhost',
 'http://localhost:4200',
 'http://localhost:3000',
 'http://localhost:8080',
 'https://localhost',
 'https://localhost:4200',
 'https://localhost:3000',
 'https://localhost:8080',
 'http://dev.grid.openmined.org',
 'https://stag.grid.openmined.org',
 'https://grid.openmined.org']

In [57]:
# for k, v in env_vars.items():
#     print(f"{k}={v}")


        "Mounts": [
            {
                "Type": "bind",
                "Source": "/Users/koen/workspace/pysyft/packages/grid/data/package-cache",
                "Destination": "/root/.cache",
                "Mode": "rw",
                "RW": true,
                "Propagation": "rprivate"
            },
            {
                "Type": "bind",
                "Source": "/Users/koen/workspace/pysyft/packages/grid/backend/grid",
                "Destination": "/app/grid",
                "Mode": "rw",
                "RW": true,
                "Propagation": "rprivate"
            },
            {
                "Type": "bind",
                "Source": "/Users/koen/workspace/pysyft/packages/syft",
                "Destination": "/app/syft",
                "Mode": "rw",
                "RW": true,
                "Propagation": "rprivate"
            },
            {
                "Type": "volume",
                "Name": "grid_credentials-data-worker",
                "Source": "/var/lib/docker/volumes/grid_credentials-data-worker/_data",
                "Destination": "/storage",
                "Driver": "local",
                "Mode": "z",
                "RW": true,
                "Propagation": ""
            }
        ],
        "Config": {
            "Hostname": "high-side",
            "Domainname": "",
            "User": "",
            "AttachStdin": false,
            "AttachStdout": true,
            "AttachStderr": true,
            "Tty": true,
            "OpenStdin": true,
            "StdinOnce": false,
            "Env": [
                "TRAEFIK_TLS_CERTS=./traefik/certs",
                "S3_ROOT_PWD=admin",
                "REDIS_STORE_DB_ID=0",
                "ASSOCIATION_TIMEOUT=10",
                "REDIS_HOST=redis",
                "DOCKER_IMAGE_WORKER=openmined/grid-enclave",
                "QUEUE_PORT=5556",
                "FRONTEND_TARGET=grid-ui-production",
                "DOCKER_IMAGE_HEADSCALE=openmined/grid-vpn-headscale",
                "DOCKER_IMAGE_TRAEFIK=traefik",
                "TRAEFIK_TAG=2b0a5be942b68dde2b00b8bca4514e4dde7f312d0d5cd397a9733bb1520f5b47",
                "SMTP_TLS=True",
                "S3_ENDPOINT=seaweedfs",
                "HTTPS_PORT=445",
                "OBLV_KEY_PATH=~/.oblv",
                "ENABLE_WARNINGS=True",
                "USE_BLOB_STORAGE=True",
                "TRAEFIK_TLS_CONF=./traefik/dynamic-configurations",
                "ENABLE_OBLV=false",
                "JAEGER_HOST=localhost",
                "DOCKER_BUILDKIT=1",
                "S3_PRESIGNED_TIMEOUT_SECS=1800",
                "RELEASE=production",
                "STACK_NAME=grid-openmined-org",
                "USERS_OPEN_REGISTRATION=False",
                "EMAILS_FROM_EMAIL=info@openmined.org",
                "PROJECT_NAME=grid",
                "SERVER_HOST=https://localhost",
                "HTTP_PORT=87",
                "ENABLE_SIGNUP=False",
                "TRAEFIK_PUBLIC_NETWORK_IS_EXTERNAL=False",
                "VERSION=0.8.2-beta.6",
                "TRAEFIK_VERSION=v2.8.1",
                "S3_ROOT_USER=admin",
                "VERSION_HASH=dockerhub",
                "MONGO_HOST=mongo",
                "DOCKER_IMAGE_BACKEND=openmined/grid-backend",
                "IGNORE_TLS_ERRORS=False",
                "HEADSCALE_PORT=8080",
                "CONTAINER_HOST=docker",
                "SMTP_USER=",
                "VITE_PUBLIC_API_BASE_URL=/api/v2",
                "DEV_MODE=True",
                "CREATE_PRODUCER=false",
                "SECRET_KEY=changethis",
                "STACK_API_KEY=zXuYOLdamjg70sB5IoVQCM5gdxGK9R79Otu9L9ZnaDZCqyeV",
                "TRACE=False",
                "SERVICE_NAME=backend",
                "MONGO_IMAGE=mongo",
                "USE_NEW_SERVICE=False",
                "JAX_ENABLE_X64=True",
                "REDIS_PORT=6379",
                "MONGO_VERSION=7.0.0",
                "TRAEFIK_PUBLIC_NETWORK=traefik-public",
                "DEFAULT_ROOT_PASSWORD=changethis",
                "NETWORK_NAME=omnet",
                "SEAWEEDFS_VERSION=3.55",
                "COMPOSE_DOCKER_CLI_BUILD=1",
                "TRAEFIK_PUBLIC_TAG=traefik-public",
                "SMTP_PASSWORD=",
                "RABBITMQ_VERSION=3",
                "REDIS_VERSION=6.2",
                "NETWORK_CHECK_INTERVAL=60",
                "DOMAIN_CHECK_INTERVAL=60",
                "DOCKER_IMAGE_SEAWEEDFS=chrislusf/seaweedfs",
                "DEFAULT_ROOT_EMAIL=info2@openmined.org",
                "NODE_SIDE_TYPE=high",
                "S3_VOLUME_SIZE_MB=1024",
                "DOCKER_IMAGE_TAILSCALE=openmined/grid-vpn-tailscale",
                "S3_PORT=8333",
                "NODE_TYPE=domain",
                "DOCKER_IMAGE_FRONTEND=openmined/grid-frontend",
                "SMTP_HOST=",
                "SMTP_PORT=587",
                "MONGO_USERNAME=root",
                "SYFT_TUTORIAL_MODE=False",
                "RELATIVE_PATH=",
                "JAEGER_PORT=14268",
                "BACKEND_STORAGE_PATH=credentials-data",
                "DOMAIN_CONNECTION_PORT=3030",
                "REDIS_LEDGER_DB_ID=1",
                "N_CONSUMERS=1",
                "BACKEND_CORS_ORIGINS=[\"http://localhost\",\"http://localhost:4200\",\"http://localhost:3000\",\"http://localhost:8080\",\"https://localhost\",\"https://localhost:4200\",\"https://localhost:3000\",\"https://localhost:8080\",\"http://dev.grid.openmined.org\",\"https://stag.grid.openmined.org\",\"https://grid.openmined.org\"]",
                "DEFAULT_ROOT_USERNAME=worker1",
                "NODE_NAME=high-side",
                "S3_REGION=us-east-1",
                "MONGO_PASSWORD=example",
                "PORT=8002",
                "DOCKER_IMAGE_VPN_IPTABLES=openmined/grid-vpn-iptables",
                "DOMAIN=localhost",
                "MONGO_PORT=27017",
                "DOCKER_IMAGE_SVELTE=openmined/grid-svelte",
                "PATH=/root/.local/bin:/usr/local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin",
                "LANG=C.UTF-8",
                "GPG_KEY=A035C8C19219BA821ECEA86B64E628F8D684696D",
                "PYTHON_VERSION=3.11.4",
                "PYTHON_PIP_VERSION=23.1.2",
                "PYTHON_SETUPTOOLS_VERSION=65.5.1",
                "PYTHON_GET_PIP_URL=https://github.com/pypa/get-pip/raw/0d8570dc44796f4369b652222cf176b3db6ac70e/public/get-pip.py",
                "PYTHON_GET_PIP_SHA256=96461deced5c2a487ddc65207ec5a9cffeca0d34e7af7ea1afc470ff0d746207",
                "PYTHONPATH=/app"
            ],
            "Cmd": [
                "/app/grid/start.sh"
            ],
            "Image": "openmined/grid-backend:0.8.2-beta.6",
            "Volumes": null,
            "WorkingDir": "/app",
            "Entrypoint": null,
            "OnBuild": null,
            "Labels": {
                "com.docker.compose.config-hash": "c14a5d85da73c7c973ad7274199b22309e5a5e2d4ac1e6ac18e52b10457af660",
                "com.docker.compose.container-number": "1",
                "com.docker.compose.depends_on": "proxy:service_started:false,backend:service_started:false",
                "com.docker.compose.image": "sha256:95c41545ecfa8667a2353d825420be76da65eb85c0dc1e85da65d1b1961a019c",
                "com.docker.compose.oneoff": "False",
                "com.docker.compose.project": "grid",
                "com.docker.compose.project.config_files": "/Users/koen/workspace/pysyft/packages/grid/docker-compose.multinode.yml,/Users/koen/workspace/pysyft/packages/grid/docker-compose.dev.yml",
                "com.docker.compose.project.working_dir": "/Users/koen/workspace/pysyft/packages/grid",
                "com.docker.compose.service": "backendworker",
                "com.docker.compose.version": "2.19.1",
                "orgs.openmined.syft": "this is a syft backend container"
            }
        },
        "NetworkSettings": {
            "Bridge": "",
            "SandboxID": "",
            "HairpinMode": false,
            "LinkLocalIPv6Address": "",
            "LinkLocalIPv6PrefixLen": 0,
            "Ports": {},
            "SandboxKey": "",
            "SecondaryIPAddresses": null,
            "SecondaryIPv6Addresses": null,
            "EndpointID": "",
            "Gateway": "",
            "GlobalIPv6Address": "",
            "GlobalIPv6PrefixLen": 0,
            "IPAddress": "",
            "IPPrefixLen": 0,
            "IPv6Gateway": "",
            "MacAddress": "",
            "Networks": {}
        }
    }
]